## TensorFlow2教程-Keras概述

Keras 是一个用于构建和训练深度学习模型的高阶 API。它可用于快速设计原型、高级研究和生产。

Keras的3个优点：
方便用户使用、模块化和可组合、易于扩展


##  1 导入tf.keras
TensorFlow2推荐使用tf.keras构建网络，常见的神经网络都包含在tf.keras.layer中(最新的tf.keras的版本可能和keras不同)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

2.8.0
2.8.0


## 2 构建简单模型
### 2.1 模型堆叠
最常见的模型类型是层的堆叠：tf.keras.Sequential 模型

In [2]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### 2.2 网络配置

tf.keras.layers中主要的网络配置参数如下：

activation：设置层的激活函数。此参数可以是函数名称字符串，也可以是函数对象。默认情况下，系统不会应用任何激活函数。

kernel_initializer 和 bias_initializer：创建层权重（核和偏置）的初始化方案。此参数是一个名称或可调用的函数对象，默认为 "Glorot uniform" 初始化器。

kernel_regularizer 和 bias_regularizer：应用层权重（核和偏置）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。

In [3]:
layers.Dense(32, activation='sigmoid')
layers.Dense(32, activation=tf.sigmoid)
layers.Dense(32, kernel_initializer='orthogonal')
layers.Dense(32, kernel_initializer=tf.keras.initializers.glorot_normal)
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01))
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l1(0.01))

## 3 训练和评估
### 3.1 设置训练流程
构建好模型后，通过调用 compile 方法配置该模型的学习流程：

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy]
)

### 3.2 输入Numpy数据
对于小型数据集，可以使用Numpy构建输入数据。

In [5]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))
val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))
model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

Epoch 1/10
10/10 [==============================] - 1s 20ms/step - loss: 11.9200 - categorical_accuracy: 0.0940 - val_loss: 12.1965 - val_categorical_accuracy: 0.1150
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 12.4760 - categorical_accuracy: 0.0820 - val_loss: 13.1342 - val_categorical_accuracy: 0.1250
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 13.8033 - categorical_accuracy: 0.0880 - val_loss: 14.9594 - val_categorical_accuracy: 0.1300
Epoch 4/10
10/10 [==============================] - 0s 4ms/step - loss: 16.1865 - categorical_accuracy: 0.0860 - val_loss: 17.9014 - val_categorical_accuracy: 0.1300
Epoch 5/10
10/10 [==============================] - 0s 3ms/step - loss: 19.5113 - categorical_accuracy: 0.0860 - val_loss: 21.7283 - val_categorical_accuracy: 0.1350
Epoch 6/10
10/10 [==============================] - 0s 3ms/step - loss: 24.1679 - categorical_accuracy: 0.0850 - val_loss: 27.3839 - val_categorical_accuracy: 0.1300
Epo

### 3.3 tf.data输入数据
对于大型数据集可以使用tf.data构建训练输入。

In [6]:
datasets = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
datasets = datasets.repeat()
val_datasets = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(32)
val_datasets = val_datasets.repeat()

model.fit(datasets, epochs=10, steps_per_epoch=30, validation_data=val_datasets, validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 5ms/step - loss: 91.0570 - categorical_accuracy: 0.1000 - val_loss: 119.5591 - val_categorical_accuracy: 0.0833
Epoch 2/10
30/30 [==============================] - 0s 2ms/step - loss: 149.0371 - categorical_accuracy: 0.0940 - val_loss: 187.4500 - val_categorical_accuracy: 0.0938
Epoch 3/10
30/30 [==============================] - 0s 2ms/step - loss: 222.6545 - categorical_accuracy: 0.0983 - val_loss: 269.8352 - val_categorical_accuracy: 0.1146
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 311.5627 - categorical_accuracy: 0.0919 - val_loss: 370.2441 - val_categorical_accuracy: 0.1771
Epoch 5/10
30/30 [==============================] - 0s 2ms/step - loss: 415.1219 - categorical_accuracy: 0.1004 - val_loss: 483.1056 - val_categorical_accuracy: 0.1562
Epoch 6/10
30/30 [==============================] - 0s 2ms/step - loss: 530.7856 - categorical_accuracy: 0.1132 - val_loss: 603.1443 - val_categorical_accuracy: 

### 3.4 评估与预测
评估和预测函数：tf.keras.Model.evaluate和tf.keras.Model.predict方法，都可以使用NumPy和tf.data.Dataset构造的输入数据进行评估和预测

In [7]:
# 模型评估
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

30/30 [==============================] - 0s 1ms/step - loss: 1086.2156 - categorical_accuracy: 0.0917


[1086.215576171875, 0.09166666865348816]

In [9]:
# 模型预测
result = model.predict(test_x, batch_size=32)
print(result)

[[0.         0.16240874 0.08368254 ... 0.00671651 0.         0.        ]
 [0.         0.08923861 0.05535543 ... 0.00927343 0.         0.        ]
 [0.         0.15965922 0.06607018 ... 0.00557643 0.         0.        ]
 ...
 [0.         0.12314524 0.06737925 ... 0.00542385 0.         0.        ]
 [0.         0.15415026 0.08174875 ... 0.00448536 0.         0.        ]
 [0.         0.13542601 0.07227626 ... 0.00791746 0.         0.        ]]


## 4 构建复杂模型

### 4.1 函数式API
tf.keras.Sequential 模型是层的简单堆叠，无法表示任意模型。使用 Keras的函数式API可以构建复杂的模型拓扑，例如：

- 多输入模型，

- 多输出模型，

- 具有共享层的模型（同一层被调用多次），

- 具有非序列数据流的模型（例如，残差连接）。

**使用函数式 API 构建的模型具有以下特征：**

- 层实例可调用并返回张量。
- 输入张量和输出张量用于定义 tf.keras.Model 实例。
- 此模型的训练方式和 Sequential 模型一样。

In [8]:
input_x = tf.keras.Input(shape=(72, ))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)
# 构建tf.keras.Model示例
model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), 
            loss=tf.keras.losses.categorical_crossentropy,
            metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 11.6681 - accuracy: 0.0830
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 12.4798 - accuracy: 0.0890
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 18.4922 - accuracy: 0.1080
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 34.0738 - accuracy: 0.1130
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 61.9721 - accuracy: 0.1050


## 4.2 模型子类化
可以通过对tf.keras.Model进行子类化并定义自己的前向传播构建完全自定义的模型
- 在__init__方法中创建层并将他们设置为类示例的属性
- 在__call___方法中定义前向传播

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义网络层
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs):
        # 定义前向传播
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    def compute_output_shape(self, input_shape):
        # 计算输出的shape
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

# 实例化模型类并训练
model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
            loss=tf.keras.losses.categorical_crossentropy,
            metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=16, epochs=5)

Epoch 1/5
63/63 [==============================] - 0s 1ms/step - loss: 14.7771 - accuracy: 0.0930
Epoch 2/5
63/63 [==============================] - 0s 1ms/step - loss: 21.9342 - accuracy: 0.0940
Epoch 3/5
63/63 [==============================] - 0s 1ms/step - loss: 26.3706 - accuracy: 0.0940
Epoch 4/5
63/63 [==============================] - 0s 1ms/step - loss: 29.8917 - accuracy: 0.1000
Epoch 5/5
63/63 [==============================] - 0s 1ms/step - loss: 33.7799 - accuracy: 0.1000


## 4.3 自定义层
通过对tf.keras.layers.Layer进行子类化并实现以下方法来创建自定义层
- __init__: 定义该层要使用的子层
- build: 创建层的权重,使用add_weight方式添加权重
- call: 定义前向传播
- compute_output_shape: 指定在给定输入形状的情况下如何计算层的输出形状
- 可选,可以通过实现get_config和from_config类方法序列化层

In [11]:
from pickletools import optimize


class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
    
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape, 
                                    initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
# 使用自定义网络层构建模型
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
            loss=tf.keras.losses.categorical_crossentropy,
            metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=16, epochs=5)
model

Epoch 1/5
63/63 [==============================] - 0s 1ms/step - loss: 11.4863 - accuracy: 0.0880
Epoch 2/5
63/63 [==============================] - 0s 1ms/step - loss: 11.4852 - accuracy: 0.0840
Epoch 3/5
63/63 [==============================] - 0s 1ms/step - loss: 11.4847 - accuracy: 0.0810
Epoch 4/5
63/63 [==============================] - 0s 939us/step - loss: 11.4841 - accuracy: 0.0890
Epoch 5/5
63/63 [==============================] - 0s 1ms/step - loss: 11.4834 - accuracy: 0.0850


## 4.4 回调
回调是传递给模型以自定义和扩展在训练期间的行为对象,我们可以编写自己的自定义回调函数,使用tf.keras.callbacks中的内置函数,常用的内置回调函数如下:
- tf.keras.callbacks.ModelCheckpoint：定期保存模型的检查点。
- tf.keras.callbacks.LearningRateScheduler：动态更改学习率。
- tf.keras.callbacks.EarlyStopping：验证性能停止提高时进行中断培训。
- tf.keras.callbacks.TensorBoard：使用TensorBoard监视模型的行为 。

In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
        callbacks=callbacks, validation_data=(val_x, val_y))
model

Epoch 1/5
63/63 [==============================] - 0s 5ms/step - loss: 11.4830 - accuracy: 0.0890 - val_loss: 11.6304 - val_accuracy: 0.0500
Epoch 2/5
63/63 [==============================] - 0s 2ms/step - loss: 11.4840 - accuracy: 0.0920 - val_loss: 11.6258 - val_accuracy: 0.0600
Epoch 3/5
63/63 [==============================] - 0s 2ms/step - loss: 11.4824 - accuracy: 0.0800 - val_loss: 11.6273 - val_accuracy: 0.0350
Epoch 4/5
63/63 [==============================] - 0s 2ms/step - loss: 11.4815 - accuracy: 0.0900 - val_loss: 11.6246 - val_accuracy: 0.0600
Epoch 5/5
63/63 [==============================] - 0s 2ms/step - loss: 11.4814 - accuracy: 0.0810 - val_loss: 11.6292 - val_accuracy: 0.0600


## 5 模型保存与恢复
5.1 权重保存

In [13]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [14]:
# 权重保存与重载
model.save_weights('./weights/model')
model.load_weights('./weights/model')
# 保存为h5格式
model.save_weights('./model.h5', save_format='h5')
model.load_weights('./model.h5')

5.2 保存网络结构

In [15]:
# 序列化成json
import json
import pprint
json_str = model.to_json()
pprint.pprint(json.loads(json_str))
# 从json中重建模型
fresh_model = tf.keras.models.model_from_json(json_str)
fresh_model

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'InputLayer',
                        'config': {'batch_input_shape': [None, 32],
                                   'dtype': 'float32',
                                   'name': 'dense_17_input',
                                   'ragged': False,
                                   'sparse': False}},
                       {'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'batch_input_shape': [None, 32],
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kern

In [18]:
# 保持为json格式
json_str = model.to_json()
print(json_str)
# 从yaml数据中重新构建模型
fresh_model = tf.keras.models.model_from_json(json_str)

{"class_name": "Sequential", "config": {"name": "sequential_4", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_17_input"}}, {"class_name": "Dense", "config": {"name": "dense_17", "trainable": true, "batch_input_shape": [null, 32], "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_18", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": 

5.3 保存整个模型

In [21]:
model = tf.keras.Sequential([
    layers.Dense(10, activation='softmax', input_shape=(72,)),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)
# 保存整个模型
model.save('all_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 11.4931 - accuracy: 0.1070
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 11.5729 - accuracy: 0.1010
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 11.6380 - accuracy: 0.1020
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 11.6564 - accuracy: 0.1020
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 11.6617 - accuracy: 0.1020


In [22]:
# 导入整个模型
model = tf.keras.models.load_model('all_model.h5')

## 6 将Keras用于Estimator
Estimator API用于针对分布式环境训练模型, 它使适用一些行业使用场景,例如大型数据集进行分布式训练并导出模型用以生产

In [23]:
model = tf.keras.Sequential([
    layers.Dense(10, activation='softmax'),
    layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hongyang.w\\AppData\\Local\\Temp\\tmpf_cjd9y9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replica

## 7 Eager execution
Eager execution是一个动态执行的编程环境，它可以立即评估操作。Keras不需要此功能，但它受tf.keras程序支持和对检查程序和调试有用。

所有的tf.keras模型构建API都与Eager execution兼容。尽管可以使用Sequential和函数API，但Eager execution有利于模型子类化和构建自定义层：其要求以代码形式编写前向传递的API（而不是通过组装现有层来创建模型的API）。

## 8 多GPU上运行
tf.keras模型可使用tf.distribute.Strategy在多个GPU上运行 。该API在多个GPU上提供了分布式培训，几乎无需更改现有代码。

当前tf.distribute.MirroredStrategy是唯一受支持的分发策略。MirroredStrategy在单台计算机上使用全缩减进行同步训练来进行图内复制。要使用 distribute.Strategys，请将优化器实例化以及模型构建和编译嵌套在Strategys中.scope()，然后训练模型。

以下示例tf.keras.Model在单个计算机上的多GPU分配。

首先，在分布式策略范围内定义一个模型：

In [24]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.SGD(0.2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 16)                176       
                                                                 
 dense_26 (Dense)            (None, 1)                 17        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 然后像单gpu一样在数据上训练模型即可
x = np.random.random((1024, 10))
y = np.random.randint(2, size=(1024, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32)
model.fit(dataset, epochs=1)

32/32 [==============================] - 0s 1ms/step - loss: 0.6988
